In [1]:
from google.colab import files

uploaded = files.upload()

In [ ]:
!pip install kaggle

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d vishweshsalodkar/customer-feedback-dataset

customer-feedback-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip customer-feedback-dataset.zip

In [ ]:
import pandas as pd

df = pd.read_csv('sentiment-analysis.csv')

df.head()

In [ ]:
df = df['Text, Sentiment, Source, Date/Time, User ID, Location, Confidence Score'].str.split(',', expand=True)
df.columns = ['Comment', 'Sentiment', 'CategoryID', 'Timestamp', 'UserID', 'Location', 'SubmissionID']



---


For simplicity of this faux scenario, I will keep the comments the same (irrelevant to feedback about an educational app) and utilize the Sentiment column more strongly to pretend it is application feedback.

Additionally, for the intention of this project being Data Engineering oriented, I will remove the Confidence Score and ignore that the Sentiment classification may be incorrect.





---



In [ ]:
df['CategoryID'] = pd.factorize(df['CategoryID'])[0]
df['SubmissionID'] = range(1, len(df) + 1)

In [ ]:
categories = ['General Comments', 'User Experience Comments', 'Algebra', 'English', 'Chemistry', 'Biology', 'Computer Science', 'Writing', 'Statistics', 'Geometry', 'Calculus', 'Physics', 'History', 'Reading', 'Economics', 'AP Maths', 'AP Sciences', 'AP English', 'Spanish', 'French', 'Germen', 'Chinese', 'Arabic', 'Geography', 'Pre-Calc', 'Philosophy', 'Music', 'AP Philosophy']

# Create a dictionary to map numbers to descriptions
mapping = {i: subj for i, subj in enumerate(categories)}

# Add a new column with descriptions based on numbers
df['Categories'] = df['CategoryID'].map(mapping)



---

Creating a faux first name, last name and email for each unique UserID


---



In [ ]:
!pip install names

In [ ]:
import names
import random

emailProviders = ['gmail.com', 'yahoo.com', 'outlook.com']
users = {'i':df['UserID'].unique(), 'First Name':[], 'Last Name':[], 'Email':[]}
for i in range(len(df['UserID'].unique())):
  users['First Name'].append(names.get_first_name())
  users['Last Name'].append(names.get_last_name())
  users['Email'].append(users['First Name'][i][0]+'.'+users['Last Name'][i]+str(random.randint(0, 99))+'@'+emailProviders[random.randint(0,2)])

userDF = pd.DataFrame(users).set_index('i')

df['First Name'] = df['UserID'].map(userDF['First Name'])
df['Last Name'] = df['UserID'].map(userDF['Last Name'])
df['Email'] = df['UserID'].map(userDF['Email'])

df.head(15)



---
The following is to match the format of the SQL Database UserFeedbackDB


---






In [ ]:
df.to_csv('customer_feedback.csv', index=False)